# Project IRIS: Sprint 1 - Data Fusion

**Objective:** This notebook performs the final step of Sprint 1: fusing the certified INFOBRAS and SINADEF datasets. The output will be a single, master DataFrame containing both G-Factors and S-Factors at the district level (`ubigeo`), ready for the feature engineering and modeling phases.

In [4]:
import pandas as pd
import numpy as np

print("--- Step 1: Loading Certified Datasets ---")

try:
    # --- LA CORRECCIÓN ESTÁ AQUÍ ---
    # Creamos una lista de las columnas que sabemos que son fechas
    date_columns_infobras = [
        'fecha_de_inicio_de_obra',
        'fecha_de_finalizacion_real',
        'fecha_finalizacion_programada_de_obra',
        'fecha_finalizacion_reprogramada_de_obra'
        # Añade otras columnas de fecha si las vas a usar
    ]

    # Le decimos a Pandas que parsee estas columnas como fechas al momento de la carga
    df_infobras = pd.read_csv(
        '../data/infobras_certificado_v_final.csv', 
        low_memory=False,
        parse_dates=date_columns_infobras # ¡La instrucción clave!
    )
    print(f"Successfully loaded INFOBRAS data: {df_infobras.shape[0]} rows.")

    # Verificamos que los tipos de datos son correctos
    print("\nVerifying dtypes for key date columns in INFOBRAS:")
    print(df_infobras[date_columns_infobras].dtypes)

    # Cargamos los datos de SINADEF (sin cambios)
    df_sinadef = pd.read_csv('../data/sinadef_certified_v1.csv')
    print(f"\nSuccessfully loaded SINADEF data: {df_sinadef.shape[0]} districts.")

except FileNotFoundError as e:
    print(f"CRITICAL ERROR: A required certified file was not found.")
    print(e)

--- Step 1: Loading Certified Datasets ---
Successfully loaded INFOBRAS data: 132137 rows.

Verifying dtypes for key date columns in INFOBRAS:
fecha_de_inicio_de_obra                    datetime64[ns]
fecha_de_finalizacion_real                 datetime64[ns]
fecha_finalizacion_programada_de_obra      datetime64[ns]
fecha_finalizacion_reprogramada_de_obra    datetime64[ns]
dtype: object

Successfully loaded SINADEF data: 2268 districts.


In [6]:
print("\n--- Step 2: Aggregating INFOBRAS Data to District Level (G-Factor Creation) ---")

# Create the 'ubigeo' key in the INFOBRAS dataset
df_infobras['ubigeo'] = df_infobras['departamento'] + '_' + df_infobras['provincia'] + '_' + df_infobras['distrito']

# --- Engineer the G-Factor metrics at the individual project level ---
df_infobras['fue_paralizado'] = (df_infobras['causal_de_paralizacion'] != 'No Paralizada').astype(int)
df_infobras['plazo_real_dias'] = (df_infobras['fecha_de_finalizacion_real'] - df_infobras['fecha_de_inicio_de_obra']).dt.days

# --- CÁLCULOS BLINDADOS ---
# Reemplazamos los ceros en los denominadores con NaN para evitar la división por cero.
# El NaN será ignorado automáticamente por la agregación .mean().

# Blindaje para ratio_sobretiempo
denominador_tiempo = df_infobras['plazo_de_ejecucion_en_dias'].replace(0, np.nan)
df_infobras['ratio_sobretiempo'] = (df_infobras['plazo_real_dias'] - df_infobras['plazo_de_ejecucion_en_dias']) / denominador_tiempo

# Blindaje para ratio_sobrecosto
denominador_costo = df_infobras['monto_viable/aprobado'].replace(0, np.nan)
df_infobras['ratio_sobrecosto'] = (df_infobras['costo_de_la_obra_en_soles'] - df_infobras['monto_viable/aprobado']) / denominador_costo


# --- Aggregate by 'ubigeo' ---
# La función .mean() ignora los valores NaN por defecto, por lo que el promedio será correcto.
infobras_aggregated_df = df_infobras.groupby('ubigeo').agg(
    g_factor_total_obras=('codigo_infobras', 'count'),
    g_factor_tasa_paralizacion=('fue_paralizado', 'mean'),
    g_factor_ratio_sobretiempo_promedio=('ratio_sobretiempo', 'mean'),
    g_factor_ratio_sobrecosto_promedio=('ratio_sobrecosto', 'mean')
).reset_index()

print("INFOBRAS data successfully aggregated by district (with anti-infinity protection).")
display(infobras_aggregated_df.head())


--- Step 2: Aggregating INFOBRAS Data to District Level (G-Factor Creation) ---
INFOBRAS data successfully aggregated by district (with anti-infinity protection).


,ubigeo,g_factor_total_obras,g_factor_tasa_paralizacion,g_factor_ratio_sobretiempo_promedio,g_factor_ratio_sobrecosto_promedio
0,AMAZONAS_BAGUA_ARAMANGO,552,0.065217,0.689552,-0.963116
1,AMAZONAS_BAGUA_BAGUA,118,0.025424,0.888986,-0.964240
2,AMAZONAS_BAGUA_COPALLIN,40,0.000000,0.442342,-1.000000
3,AMAZONAS_BAGUA_EL PARCO,16,0.000000,0.612704,-1.000000
4,AMAZONAS_BAGUA_IMAZA,254,0.118110,1.810063,-0.915809


In [7]:
# --- Step 3: Fusing INFOBRAS and SINADEF Data ---

print("\nIniciando la fusión de los datasets de INFOBRAS y SINADEF...")

# Usamos un 'left merge' (unión por la izquierda).
# Esto es una decisión estratégica importante. Significa que:
# 1. Mantenemos TODOS los distritos que tienen datos de obras (nuestro universo principal).
# 2. Añadimos los datos de salud (S-Factors) a esos distritos SI existen.
# 3. Si un distrito tiene obras pero no tiene datos de SINADEF, los S-Factors aparecerán como NaN (Nulo).
df_fused = pd.merge(infobras_aggregated_df, df_sinadef, on='ubigeo', how='left')

print("¡Fusión de datos completada!")
print(f"Dimensiones del DataFrame fusionado final: {df_fused.shape}")

# --- Auditoría Post-Fusión ---
# Verificamos cuántos distritos con obras no tuvieron una contraparte en los datos de SINADEF.
nulos_sinadef_post_merge = df_fused['s_factor_total_muertes'].isna().sum()
total_distritos_infobras = len(df_fused)
print(f"\nSe encontraron {nulos_sinadef_post_merge} de {total_distritos_infobras} distritos de INFOBRAS sin datos de salud correspondientes.")

print("\n--- Muestra del DataFrame Fusionado ---")
# Mostramos columnas de ambos datasets para verificar la unión
columnas_a_mostrar = [
    'ubigeo', 
    'g_factor_total_obras', 
    'g_factor_ratio_sobrecosto_promedio', 
    's_factor_total_muertes', 
    's_factor_edad_prom_muerte'
]
display(df_fused[columnas_a_mostrar].sample(10, random_state=42))


Iniciando la fusión de los datasets de INFOBRAS y SINADEF...
¡Fusión de datos completada!
Dimensiones del DataFrame fusionado final: (2017, 8)

Se encontraron 228 de 2017 distritos de INFOBRAS sin datos de salud correspondientes.

--- Muestra del DataFrame Fusionado ---


,ubigeo,g_factor_total_obras,g_factor_ratio_sobrecosto_promedio,s_factor_total_muertes,s_factor_edad_prom_muerte
1555,LIMA_YAUYOS_OMAS,28,-1.000000,24.0,74.916667
526,AYACUCHO_LUCANAS_CABANA,34,-0.965420,42.0,80.023810
393,AREQUIPA_CAMANA_QUILCA,18,-1.000000,37.0,60.027027
1788,PUNO_CARABAYA_OLLACHEA,58,-0.971362,159.0,48.968553
433,AREQUIPA_CAYLLOMA_HUAMBO,25,-0.945368,22.0,71.818182
1159,JUNIN_CONCEPCION_MATAHUASI,47,-0.952157,220.0,67.977273
1090,HUANUCO_YAROWILCA_OBAS,25,-1.000000,162.0,65.537037
429,AREQUIPA_CAYLLOMA_CHAMACA,1,-1.000000,NaN,NaN
1801,PUNO_EL COLLAO_PILCUYO,81,-1.000000,558.0,73.783154
530,AYACUCHO_LUCANAS_CHULUCANAS,7,-0.570244,NaN,NaN


In [8]:
# --- Step 4: Exporting the Master Dataset ---
master_path = '../data/iris_master_dataset_v1.csv'
df_fused.to_csv(master_path, index=False)

print("\n----------------------------------------------------")
print("✅  ¡SPRINT 1 (JUEVES) COMPLETADO CON ÉXITO!")
print(f"El dataset maestro '{master_path}' ha sido exportado.")
print("Este archivo contiene la base de datos consolidada para el Proyecto IRIS.")


----------------------------------------------------
✅  ¡SPRINT 1 (JUEVES) COMPLETADO CON ÉXITO!
El dataset maestro '../data/iris_master_dataset_v1.csv' ha sido exportado.
Este archivo contiene la base de datos consolidada para el Proyecto IRIS.


In [7]:
# --- Preparación Final para el Dashboard ---

# ¡LA CORRECCIÓN ESTÁ AQUÍ! Importamos la librería pandas
import pandas as pd

# Cargamos nuestros scores
try:
    df_scores = pd.read_csv('../data/iris_scores_v1.csv')
    print("Archivo de scores cargado con éxito.")
except FileNotFoundError:
    print("Error: No se encontró 'data/iris_scores_v1.csv'. Asegúrate de que el archivo existe.")

# (Aquí continuaría el resto del código para la fusión y exportación)
# Ejemplo:
# df_ubigeo['departamento'] = df_ubigeo['DEPARTAMENTO'].str.upper().str.strip()
# ... etc ...
# df_ubigeo['ubigeo_texto'] = df_ubigeo['departamento'] + '_' + df_ubigeo['provincia'] + '_' + df_ubigeo['distrito']
# df_scores_final = pd.merge(df_scores, df_ubigeo[['ubigeo_texto', 'UBIGEO']], left_on='ubigeo', right_on='ubigeo_texto', how='left')
# df_scores_final.to_csv('data/iris_scores_for_dashboard.csv', index=False)
# print("Archivo de scores preparado para el dashboard y exportado.")

Archivo de scores cargado con éxito.


In [10]:
# Cargamos la tabla de mapeo de UBIGEO
try:
    df_ubigeo = pd.read_csv('../data/ubigeo_peru_2016_distritos.csv')
    print("Tabla de mapeo UBIGEO cargada con éxito.")
except FileNotFoundError:
    print("Error: No se encontró 'data/ubigeo_peru.csv'. Asegúrate de haber descargado este archivo.")

Tabla de mapeo UBIGEO cargada con éxito.


In [17]:
# --- 1. Cargar TODOS los datasets necesarios (CON LOS NOMBRES CORRECTOS) ---
try:
    df_scores = pd.read_csv('../data/iris_scores_v1.csv')
    
    # ¡LA CORRECCIÓN ESTÁ AQUÍ! Usamos los nombres de archivo exactos que descargaste.
    df_departamentos = pd.read_csv('../data/ubigeo_peru_2016_departamentos.csv', dtype=str)
    df_provincias = pd.read_csv('../data/ubigeo_peru_2016_provincias.csv', dtype=str)
    # Asumimos que el de distritos también sigue este patrón
    df_distritos = pd.read_csv('../data/ubigeo_peru_2016_distritos.csv', dtype=str)
    
    print("Todos los datasets han sido cargados con éxito.")
    datos_cargados = True
except FileNotFoundError as e:
    print(f"Error: No se pudo encontrar un archivo. Asegúrate de que los nombres de los archivos CSV de UBIGEO son correctos.")
    print(e)
    datos_cargados = False


Todos los datasets han sido cargados con éxito.


In [21]:
# --- 2. Reconstruir la Tabla Maestra de UBIGEO (Versión a Prueba de Errores) ---
print("\n--- Reconstruyendo la jerarquía de UBIGEO ---")

# Renombramos las columnas
df_departamentos = df_departamentos.rename(columns={'id': 'department_id', 'name': 'departamento'})
df_provincias = df_provincias.rename(columns={'id': 'province_id', 'name': 'provincia'})
df_distritos = df_distritos.rename(columns={'id': 'ubigeo_code', 'name': 'distrito'})

# --- VERIFICACIÓN 1: Columnas de df_provincias ---
print("\nColumnas en la tabla de Provincias (df_provincias):")
print(df_provincias.columns.tolist())

# --- PRIMERA FUSIÓN: Distritos + Provincias ---
# Creamos la clave 'province_id' en los distritos a partir del 'ubigeo_code'
if 'province_id' not in df_distritos.columns:
    df_distritos['province_id'] = df_distritos['ubigeo_code'].str[:4]

print("\nRealizando la primera fusión (Distritos + Provincias)...")
df_geo = pd.merge(df_distritos, df_provincias, on='province_id', how='left')

# --- VERIFICACIÓN 2: Columnas DESPUÉS de la primera fusión ---
print("\nColumnas en la tabla intermedia (df_geo) después de unir con Provincias:")
print(df_geo.columns.tolist())

# --- SEGUNDA FUSIÓN: (Distritos+Provincias) + Departamentos ---
# Ahora, comprobamos explícitamente si 'department_id' existe antes de hacer el merge.
if 'department_id' in df_geo.columns:
    print("\n'department_id' encontrado. Realizando la segunda fusión...")
    df_geo = pd.merge(df_geo, df_departamentos, on='department_id', how='left')

    # Seleccionamos y ordenamos las columnas finales
    df_geo = df_geo[['ubigeo_code', 'distrito', 'provincia', 'departamento']]

    print("\nTabla maestra de UBIGEO creada con éxito.")
    display(df_geo.head())
else:
    print("\n¡ERROR DE DIAGNÓSTICO! La columna 'department_id' no se añadió a df_geo después de la primera fusión.")
    print("Por favor, revisa la estructura del archivo 'ubigeo_peru_2016_provincias.csv'.")


--- Reconstruyendo la jerarquía de UBIGEO ---

Columnas en la tabla de Provincias (df_provincias):
['province_id', 'provincia', 'department_id']

Realizando la primera fusión (Distritos + Provincias)...

Columnas en la tabla intermedia (df_geo) después de unir con Provincias:
['ubigeo_code', 'distrito', 'province_id', 'department_id_x', 'provincia', 'department_id_y']

¡ERROR DE DIAGNÓSTICO! La columna 'department_id' no se añadió a df_geo después de la primera fusión.
Por favor, revisa la estructura del archivo 'ubigeo_peru_2016_provincias.csv'.


In [24]:
# --- 2. Reconstruir la Tabla Maestra de UBIGEO (Versión Anti-Conflicto) ---
print("\n--- Reconstruyendo la jerarquía de UBIGEO ---")

# Renombramos las columnas en cada tabla
df_departamentos = df_departamentos.rename(columns={'id': 'department_id', 'name': 'departamento'})
df_provincias = df_provincias.rename(columns={'id': 'province_id', 'name': 'provincia'})
df_distritos = df_distritos.rename(columns={'id': 'ubigeo_code', 'name': 'distrito'})

# --- PREVENCIÓN DEL CONFLICTO ---
# Eliminamos la columna 'department_id' de la tabla de distritos para evitar el conflicto de nombres en el merge.
if 'department_id' in df_distritos.columns:
    df_distritos = df_distritos.drop(columns=['department_id'])

# --- PRIMERA FUSIÓN: Distritos + Provincias ---
# Ahora no habrá conflicto y la columna 'department_id' de las provincias se añadirá limpiamente.
df_geo = pd.merge(df_distritos, df_provincias, on='province_id', how='left')

# --- SEGUNDA FUSIÓN: (Distritos+Provincias) + Departamentos ---
# Como 'df_geo' ahora tiene una única y correcta columna 'department_id', este merge funcionará.
df_geo = pd.merge(df_geo, df_departamentos, on='department_id', how='left')

# --- SELECCIÓN FINAL DE COLUMNAS ---
# Seleccionamos y ordenamos las columnas que nos interesan
df_geo = df_geo[['ubigeo_code', 'distrito', 'provincia', 'departamento']]

print("Tabla maestra de UBIGEO creada con éxito.")
display(df_geo.head())


--- Reconstruyendo la jerarquía de UBIGEO ---
Tabla maestra de UBIGEO creada con éxito.


,ubigeo_code,distrito,provincia,departamento
0,010101,Chachapoyas,Chachapoyas,Amazonas
1,010102,Asunción,Chachapoyas,Amazonas
2,010103,Balsas,Chachapoyas,Amazonas
3,010104,Cheto,Chachapoyas,Amazonas
4,010105,Chiliquin,Chachapoyas,Amazonas


In [26]:
# --- 3. Limpieza y Estandarización de AMBAS tablas para la fusión ---
print("\n--- Estandarizando nombres para la fusión (el paso más delicado) ---")

# (Todo el código de estandarización y merge que ya funcionó va aquí...)
# ...
df_scores['distrito_norm'] = df_scores['ubigeo'].str.split('_').str[2].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper().str.strip()

print("Ambas tablas han sido estandarizadas para la fusión.")

# --- 4. La Fusión Final ---
print("\n--- Realizando la fusión para añadir el código UBIGEO ---")
df_scores_final = pd.merge(
    df_scores,
    df_geo,
    left_on=['departamento_norm', 'provincia_norm', 'distrito_norm'],
    right_on=['departamento', 'provincia', 'distrito'],
    how='left'
)

# --- 5. Verificación y Limpieza Post-Fusión ---
coincidencias = df_scores_final['ubigeo_code'].notna().sum()
total = len(df_scores_final)
print(f"\nSe encontraron coincidencias de UBIGEO para {coincidencias} de {total} distritos ({coincidencias/total:.2%}).")

df_scores_final = df_scores_final.drop(columns=['departamento_norm', 'provincia_norm', 'distrito_norm', 'departamento', 'provincia', 'distrito'])


# --- 6. Exportación (CON RUTA CORREGIDA) ---
# ¡LA CORRECCIÓN ESTÁ AQUÍ! Añadimos '../' para subir un nivel.
ruta_final = '../data/iris_scores_for_dashboard.csv'
df_scores_final.to_csv(ruta_final, index=False)

print(f"\n✅ Archivo '{ruta_final}' exportado con éxito.")
print("Este archivo está listo para ser usado en la aplicación de Streamlit.")
display(df_scores_final.head())


--- Estandarizando nombres para la fusión (el paso más delicado) ---
Ambas tablas han sido estandarizadas para la fusión.

--- Realizando la fusión para añadir el código UBIGEO ---

Se encontraron coincidencias de UBIGEO para 1778 de 2017 distritos (88.15%).

✅ Archivo '../data/iris_scores_for_dashboard.csv' exportado con éxito.
Este archivo está listo para ser usado en la aplicación de Streamlit.


,ubigeo,g_factor_tasa_paralizacion,g_factor_ratio_sobretiempo_promedio,g_factor_ratio_sobrecosto_promedio,s_factor_total_muertes,s_factor_edad_prom_muerte,s_factor_tasa_prevenibles,g_score,s_score,iris_score,ubigeo_code
0,AMAZONAS_BAGUA_ARAMANGO,0.065217,0.039143,0.000403,0.004811,0.470946,0.0,0.034921,0.158586,0.096753,010202
1,AMAZONAS_BAGUA_BAGUA,0.025424,0.045799,0.000391,0.019787,0.411892,0.0,0.023871,0.143893,0.083882,010201
2,AMAZONAS_BAGUA_COPALLIN,0.000000,0.030893,0.000000,0.002372,0.421218,0.0,0.010298,0.141197,0.075747,010203
3,AMAZONAS_BAGUA_EL PARCO,0.000000,0.036578,0.000000,0.001016,0.408680,0.0,0.012193,0.136565,0.074379,010204
4,AMAZONAS_BAGUA_IMAZA,0.118110,0.076538,0.000920,0.002575,0.660282,0.0,0.065189,0.220952,0.143071,010205
